In [1]:
import MySQLdb
import os
import warnings
import configparser
import csv
import sys
csv.field_size_limit(sys.maxsize)
config = configparser.ConfigParser()
#config.read('/usr/local/airflow/PatentsView-DB/Development/config.ini')
config.read('config.ini')
host = config['AWS']['HOST']
username = config['AWS']['USERNAME']
password = config['AWS']['PASSWORD']
temporary_upload_db = config['AWS']['TEMPORARY_UPLOAD_DB']

patent_schema_sql = config['LOCAL']['PATENT_SCHEMA_PATH_TEMP']
data_output_dir = config['LOCAL']['OUTFILE_FOLDER_TEMP']

In [2]:
#you will need to adapt this to read the information from the new config file
#which is formatted differently
def connect_to_db():
    mydb = MySQLdb.connect(host=host,
    user=username,
    passwd=password,
    local_infile = 1)
    
    return mydb


#this chunk creates a new empty sql schema for the new database which is going to get uploaded
warnings.filterwarnings('ignore', category = MySQLdb.Warning)
mydb = connect_to_db()
cursor = mydb.cursor()
cursor.execute("create schema " + temporary_upload_db + ";")
cursor.execute("use " + temporary_upload_db + ";")


0

In [3]:
#this is a bit hacky, you are welcome to fix it if you can think of a better way, but don't spend more than 15 minutes on it
with open(patent_schema_sql, 'r') as f:
    commands = f.read().replace('\n', '').split(';')[:-1]
    for command in commands:
        cursor.execute(command)
    mydb.commit()
    #cursor.close()

In [4]:
def convert_to_utf(in_file_path, out_file_path):
    """
    read in a csv file and output another csv file with same content but 
    encode utf-8
    """
    with open(in_file_path, 'r', encoding='utf-8', errors='ignore') as infile, open(out_file_path, 'w') as outfile:
        inputs = csv.reader(infile)
        output = csv.writer(outfile)

        for index, row in enumerate(inputs):
             # Create file with no header
            if index == 0:
                continue
            output.writerow(row)

In [5]:
folders = os.listdir(data_output_dir)
folder_lst = [folder for folder in folders if folder != '.DS_Store']

#convert each csv file to unicode utf-8 (new file name should be old_encode.csv)
# clearly this is not actually converting to utf-8 for some reason
for folder in folder_lst:
    folder = data_output_dir + '/' + folder
    fields = [item for item in os.listdir(folder) if not item in ['error_counts.csv', 'error_data.csv']]
    for f in fields:
        in_file_path = folder + '/' + f
        out_file_path = folder + '/' + f.split('.')[0] + "_encode.csv"
        convert_to_utf(in_file_path, out_file_path)

In [13]:
#then we need to upload the data into these temporary tables
#if you run into encoding errors please let Sarah know right away, because I have ideas
# NOTE: if you are testing on data from H:\share\Science Policy Portfolio\PatentsView IV\ParsedData\180515
#  detail_desc_text.csv is the first file with encoding problem
for folder in folder_lst:
    folder = data_output_dir + '/' + folder
    fields = [item for item in os.listdir(folder) if not item in ['error_counts.csv', 'error_data.csv']]
    fields_encode = [item for item in fields if item.endswith('_encode.csv')]
    for f in fields_encode:
        print(f)
        cursor.execute("load data local infile '{0}/{1}' ignore into table {2} CHARACTER SET utf8 fields terminated by '\t' lines terminated by '\r\n' ignore 1 lines".format(folder, f, f.replace("_encode.csv", "")))

mainclass_encode.csv
pct_data_encode.csv
subclass_encode.csv
foreign_priority_encode.csv
claim_encode.csv
foreigncitation_encode.csv
usapplicationcitation_encode.csv
rawassignee_encode.csv
uspc_encode.csv
draw_desc_text_encode.csv
application_encode.csv
rel_app_text_encode.csv
rawlawyer_encode.csv
non_inventor_applicant_encode.csv
usreldoc_encode.csv
government_interest_encode.csv
us_term_of_grant_encode.csv
detail_desc_text_encode.csv


OperationalError: (1300, "Invalid utf8 character string: 'As will be seen from the disclosure herein, certain embodiments '")